In [1]:
import up42

In [2]:
# Authenticate and initialize project
up42.authenticate(project_id="b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4", 
                  project_api_key="R9MCBBUQ.5fdXwrbfTkcPbwwaRWJVpdZQuQ9oSEsotlX")
project = up42.initialize_project()

2022-05-11 15:55:22,169 - Authentication with UP42 successful!
2022-05-11 15:55:23,119 - Initialized Project(name: Forestfire, project_id: b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4, description: , createdAt: 2022-05-11T19:30:23.588443Z)


In [3]:
# Define the aoi 
namib_desert_aoi = {"type": "FeatureCollection","features": [{"type": "Feature","properties": {},
      "geometry": {"type": "Polygon","coordinates": [[[14.610298,-24.516841],
                                                      [14.622478,-24.516841],
                                                      [14.622478,-24.50833],
                                                      [14.610298,-24.50833],
                                                      [14.610298,-24.516841]]]}}]}

In [4]:
# Construct the workflow
workflow = project.create_workflow(name="Landsat8-Geotiff-example", use_existing=True)
input_tasks = ["sentinelhub-landsat8-aoiclipped"]
workflow.add_workflow_tasks(input_tasks)

2022-05-11 15:55:36,566 - Getting existing workflows in project ...
2022-05-11 15:55:38,350 - Got 2 workflows for project b2ee0f3d-60ae-4d6e-859e-a1f705d3f3b4.
2022-05-11 15:55:39,449 - Using existing workflow: Landsat8-Geotiff-example - c25aa270-4030-4069-a6ef-5071f3091884
2022-05-11 15:55:42,247 - Added tasks to workflow: [{'name': 'sentinelhub-landsat8-aoiclipped:1', 'parentName': None, 'blockId': '95519b2d-09d7-4cd0-a321-4d6a46bef6c1'}]


In [5]:
# Define the aoi and input parameters of the workflow to run it
input_parameters = workflow.construct_parameters(geometry=namib_desert_aoi, 
                                                 geometry_operation='bbox', 
                                                 start_date="2018-11-07",
                                                 end_date="2020-12-31")
input_parameters["sentinelhub-landsat8-aoiclipped:1"].update({"max_cloud_cover":100})  

In [6]:
# Estimate parameters
workflow.estimate_job(input_parameters)

2022-05-11 15:56:05,162 - Estimated: 1-2 Credits, Duration: 0-0 min.


{'sentinelhub-landsat8-aoiclipped:1': {'blockConsumption': {'resources': {'unit': 'MEGAPIXEL',
    'min': 0,
    'max': 0.056294},
   'credit': {'min': 0, 'max': 1}},
  'machineConsumption': {'duration': {'min': 0, 'max': 0},
   'credit': {'min': 1, 'max': 1}}}}

In [7]:
# Run the actual live job
job = workflow.run_job(input_parameters, track_status=True)

2022-05-11 15:56:15,040 - Selected input_parameters: {'sentinelhub-landsat8-aoiclipped:1': {'time': '2018-11-07T00:00:00Z/2020-12-31T23:59:59Z', 'limit': 1, 'zoom_level': 12, 'max_cloud_cover': 100, 'bbox': [14.610298, -24.516841, 14.622478, -24.50833]}}
2022-05-11 15:56:18,950 - Created and running new job: 55bb3b5a-e587-46f8-83ef-c4d5bd57a84c.
2022-05-11 15:56:19,269 - Tracking job status continuously, reporting every 30 seconds...
2022-05-11 15:56:52,064 - Job is RUNNING! - 55bb3b5a-e587-46f8-83ef-c4d5bd57a84c
2022-05-11 15:56:57,483 - Job is FAILED! - 55bb3b5a-e587-46f8-83ef-c4d5bd57a84c - Printing logs ...
2022-05-11 15:56:57,484 - Getting job tasks: 55bb3b5a-e587-46f8-83ef-c4d5bd57a84c
2022-05-11 15:56:57,942 - Getting logs for 1 job tasks: ['592d3d05-9c60-4be6-94d8-cbd685a8d7e8']


Printing logs of 1 JobTasks in Job with job_id 55bb3b5a-e587-46f8-83ef-c4d5bd57a84c:

----------------------------------------------------------
JobTask 1 with jobtask_id 592d3d05-9c60-4be6-94d8-cbd685a8d7e8:

2022-05-11 19:56:28,541 - blockutils.common - DEBUG - Raw task parameters from UP42_TASK_PARAMETERS are: {"bbox":[14.610298,-24.516841,14.622478,-24.50833],"time":"2018-11-07T00:00:00Z/2020-12-31T23:59:59Z","limit":1,"zoom_level":12,"max_cloud_cover":100}
2022-05-11 19:56:28,541 - landsat8 - INFO - Identifying needed tiles to satisfy request
2022-05-11 19:56:28,542 - landsat8 - DEBUG - Found 1 potential tiles in image
2022-05-11 19:56:28,542 - landsat8 - INFO - Fetching available image dates from WFS
2022-05-11 19:56:28,543 - apiutils.sentinelhub - DEBUG - ?SERVICE=WFS&REQUEST=GetFeature&LAYERS=BANDS-L8&BBOX=14.610298,-24.516841,14.622478,-24.50833&FORMAT=application/json&SRSNAME=CRS:84&TYPENAMES=L8.TILE&TIME=2018-11-07/2020-12-31/P1D&MAXCC=100
2022-05-11 19:56:29,400 - landsat8 

ValueError: Job has failed! See the above log.

In [ ]:
# Download results
job.download_results()

In [ ]:
# Plot the results
job.plot_results(figsize=(6,6), bands=[4,3,2])